<a href="https://colab.research.google.com/github/amit-dhidhi-dev/Basic-Python-Programs/blob/main/video_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Video generated and upload youtube channel**

In [ ]:
# @title mount to google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title install library
# FFmpeg इंस्टॉल करें
!apt update
!apt install -y ffmpeg imagemagick

# लाइब्रेरीज इंपोर्ट करें
import os
from PIL import Image, ImageFilter, ImageDraw, ImageFont
import textwrap
import subprocess
import random

In [ ]:
# @title  create_fast_video_from_slides()
def create_fast_video_from_slides(images_list, audio_path, output_video="output.mp4", duration_per_slide=5):
    """
    FFmpeg का उपयोग करके तेजी से वीडियो बनाता है

    Parameters:
    - images_list: स्लाइड इमेज के पथ की लिस्ट
    - audio_path: ऑडियो फाइल का पथ
    - output_video: आउटपुट वीडियो का नाम
    - duration_per_slide: प्रत्येक स्लाइड की अवधि (सेकंड में)
    """

    # टेम्पोररी फाइल बनाएं जिसमें सभी इमेज और उनकी अवधि लिखी हो
    concat_file = "concat_list.txt"

    with open(concat_file, 'w') as f:
        for image in images_list:
            # duration_per_slide सेकंड के लिए प्रत्येक इमेज
            f.write(f"file '{image}'\n")
            f.write(f"duration {duration_per_slide}\n")

    # FFmpeg कमांड - यह सबसे तेज़ तरीका है
    # Hardware acceleration के लिए GPU support (अगर उपलब्ध हो)
    cmd = [
        'ffmpeg',
        '-y',  # overwrite output files without asking
        '-f', 'concat',
        '-safe', '0',
        '-i', concat_file,
        '-i', audio_path,
        '-c:v', 'libx264',  # H.264 encoding
        '-preset', 'ultrafast',  # सबसे तेज़ encoding preset
        '-crf', '23',  # क्वालिटी सेटिंग (निचला = बेहतर)
        '-pix_fmt', 'yuv420p',
        '-c:a', 'aac',
        '-b:a', '192k',
        '-shortest',  # ऑडियो/वीडियो में से जो छोटा हो उतनी ही लंबी वीडियो
        '-r', '30',  # फ्रेम रेट
        '-vf', 'fps=30,format=yuv420p',
        '-movflags', '+faststart',  # वेब स्ट्रीमिंग के लिए ऑप्टिमाइज़
        output_video
    ]

    print("वीडियो बनाना शुरू कर रहे हैं...")
    result = subprocess.run(cmd, capture_output=True, text=True)

    if result.returncode == 0:
        print(f"वीडियो सफलतापूर्वक बन गया: {output_video}")
    else:
        print(f"त्रुटि: {result.stderr}")

    # टेम्पोररी फाइल्स साफ करें
    if os.path.exists(concat_file):
        os.remove(concat_file)

    return output_video

def create_optimized_video(images_list, audio_path, output_video="output_fast.mp4", duration_per_slide=5):
    """
    और भी तेज़ वीडियो जनरेशन - parallel processing के साथ
    """

    # सबसे तेज़ FFmpeg command
    # image2 demuxer का उपयोग करके

    # पहले एक टेम्पोररी वीडियो बनाएं सिर्फ इमेज्स से
    temp_video = "temp_images.mp4"

    # image2 demuxer के साथ - यह बहुत तेज़ है
    cmd = [
        'ffmpeg',
        '-y',
        '-framerate', str(1/duration_per_slide),  # प्रति स्लाइड फ्रेम रेट
        '-pattern_type', 'glob',
        '-i', 'slide_*.jpg',  # सभी स्लाइड इमेज
        '-c:v', 'libx264',
        '-preset', 'ultrafast',
        '-crf', '22',
        '-pix_fmt', 'yuv420p',
        '-r', '30',
        temp_video
    ]

    print("इमेज्स से वीडियो बनाना शुरू...")
    result = subprocess.run(cmd, capture_output=True, text=True)

    if result.returncode != 0:
        # अगर glob काम न करे तो concat का उपयोग करें
        concat_file = "concat_list.txt"
        with open(concat_file, 'w') as f:
            for image in images_list:
                f.write(f"file '{image}'\n")
                f.write(f"duration {duration_per_slide}\n")

        cmd = [
            'ffmpeg',
            '-y',
            '-f', 'concat',
            '-safe', '0',
            '-i', concat_file,
            '-c:v', 'libx264',
            '-preset', 'ultrafast',
            '-crf', '22',
            '-pix_fmt', 'yuv420p',
            '-r', '30',
            temp_video
        ]

        result = subprocess.run(cmd, capture_output=True, text=True)
        if os.path.exists(concat_file):
            os.remove(concat_file)

    # अब ऑडियो मिलाएं
    cmd = [
        'ffmpeg',
        '-y',
        '-i', temp_video,
        '-i', audio_path,
        '-c:v', 'copy',  # वीडियो को re-encode न करें
        '-c:a', 'aac',
        '-b:a', '192k',
        '-shortest',
        '-movflags', '+faststart',
        output_video
    ]

    print("ऑडियो मिला रहे हैं...")
    result = subprocess.run(cmd, capture_output=True, text=True)

    # टेम्पोररी फाइल साफ करें
    if os.path.exists(temp_video):
        os.remove(temp_video)

    return output_video

In [ ]:
# @title
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageEnhance
import textwrap
import os


def create_slide_old(image_path, text, output_path, blur_radius=5, max_words=20, font_size=48):
    """
    (image_path, text, output_path, blur_radius=2)
    एक स्लाइड बनाता है जहां इमेज को ब्लर करके बैकग्राउंड में रखा जाता है
    और उसके ऊपर 20 शब्दों का टेक्स्ट दिखाया जाता है

    Parameters:
    -----------
    image_path : str
        इनपुट इमेज का पथ
    text : str
        पूरा टेक्स्ट जिससे 20 शब्द लेने हैं
    output_path : str
        आउटपुट स्लाइड को सेव करने का पथ
    blur_radius : int, optional
        बैकग्राउंड ब्लर की तीव्रता (डिफ़ॉल्ट: 5)
    max_words : int, optional
        प्रदर्शित करने के लिए अधिकतम शब्द (डिफ़ॉल्ट: 20)
    font_size : int, optional
        फॉन्ट का आकार (डिफ़ॉल्ट: 48)

    Returns:
    --------
    str : सेव की गई स्लाइड का पथ
    """

    # 1. पहले 20 शब्द निकालें
    words = text.split()[:max_words]
    display_text = " ".join(words)

    # 2. इमेज लोड करें
    try:
        img = Image.open(image_path)
    except Exception as e:
        print(f"इमेज लोड करने में त्रुटि: {e}")
        return None

    # 3. YouTube वीडियो साइज (1920x1080)
    youtube_size = (1920, 1080)

    # 4. इमेज को रिसाइज करें (aspect ratio बनाए रखते हुए)
    img_ratio = img.width / img.height
    target_ratio = youtube_size[0] / youtube_size[1]

    if img_ratio > target_ratio:
        # इमेज चौड़ी है, height के हिसाब से resize करें
        new_height = youtube_size[1]
        new_width = int(new_height * img_ratio)
    else:
        # इमेज लंबी है, width के हिसाब से resize करें
        new_width = youtube_size[0]
        new_height = int(new_width / img_ratio)

    img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

    # 5. Center crop करें
    left = (new_width - youtube_size[0]) // 2
    top = (new_height - youtube_size[1]) // 2
    right = left + youtube_size[0]
    bottom = top + youtube_size[1]
    img = img.crop((left, top, right, bottom))

    # 6. इमेज को ब्लर करें (बैकग्राउंड इफेक्ट)
    blurred_img = img.filter(ImageFilter.GaussianBlur(radius=blur_radius))

    # 7. ब्राइटनेस कम करें ताकि टेक्स्ट पढ़ने लायक हो
    enhancer = ImageEnhance.Brightness(blurred_img)
    darkened_img = enhancer.enhance(0.7)  # 70% brightness

    # 8. डार्क ओवरले ऐड करें
    overlay = Image.new('RGBA', youtube_size, (0, 0, 0, 120))  # Semi-transparent black
    final_img = darkened_img.convert('RGBA')
    final_img = Image.alpha_composite(final_img, overlay)

    # 9. टेक्स्ट ड्रा करने के लिए तैयार करें
    draw = ImageDraw.Draw(final_img)

    # 10. फॉन्ट लोड करें (Colab के लिए)
    try:
        # Colab में उपलब्ध फॉन्ट
        font_paths = [
            '/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf',
            '/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf',
            '/usr/share/fonts/truetype/noto/NotoSans-Bold.ttf',
            '/usr/share/fonts/truetype/msttcorefonts/Arial_Bold.ttf'
        ]

        font = None
        for font_path in font_paths:
            if os.path.exists(font_path):
                font = ImageFont.truetype(font_path, font_size)
                break

        if font is None:
            # कोई फॉन्ट नहीं मिला, default use करें
            font = ImageFont.load_default()
            print("डिफॉल्ट फॉन्ट का उपयोग किया जा रहा है")

    except Exception as e:
        print(f"फॉन्ट लोड करने में त्रुटि: {e}")
        font = ImageFont.load_default()

    # 11. टेक्स्ट को लपेटें (wrap)
    margin = 100
    max_width = youtube_size[0] - 2 * margin

    # लाइन की लंबाई कैलकुलेट करने के लिए
    avg_char_width = font_size * 0.6  # अनुमानित चौड़ाई
    chars_per_line = int(max_width / avg_char_width)

    # wrapped_text = textwrap.fill(display_text, width=chars_per_line)
    wrapped_text = textwrap.fill(text, width=chars_per_line)

    # 12. टेक्स्ट की डाइमेंशन कैलकुलेट करें
    lines = wrapped_text.split('\n')
    line_heights = []
    max_line_width = 0

    for line in lines:
        # textbbox method का उपयोग करें (नया method)
        bbox = draw.textbbox((0, 0), line, font=font)
        line_width = bbox[2] - bbox[0]
        line_height = bbox[3] - bbox[1]
        line_heights.append(line_height)
        max_line_width = max(max_line_width, line_width)

    total_height = sum(line_heights) + (10 * (len(lines) - 1))  # 10px line spacing

    # 13. टेक्स्ट की पोजीशन कैलकुलेट करें (center में)
    text_x = (youtube_size[0] - max_line_width) // 2
    text_y = (youtube_size[1] - total_height) // 2



    # 15. टेक्स्ट ड्रा करें
    current_y = text_y
    for i, line in enumerate(lines):
        draw.text((text_x, current_y), line, font=font, fill=(255, 255, 255, 255))
        current_y += line_heights[i] + 10  # Move down for next line



    # 17. स्लाइड सेव करें
    try:
        # Ensure output directory exists
        output_dir = os.path.dirname(output_path)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Convert to RGB if saving as JPEG
        if output_path.lower().endswith('.jpg') or output_path.lower().endswith('.jpeg'):
            final_img = final_img.convert('RGB')

        final_img.save(output_path, quality=95)
        print(f"✅ स्लाइड सफलतापूर्वक सेव की गई: {output_path}")
        print(f"   आकार: {youtube_size[0]}x{youtube_size[1]}")
        print(f"   शब्द: {len(words)}/{max_words}")
        print(f"   ब्लर: {blur_radius}")

        return output_path

    except Exception as e:
        print(f"स्लाइड सेव करने में त्रुटि: {e}")
        return None

In [ ]:
# @title create_slide()
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageEnhance
import textwrap
import os

def create_slide(image_path, text, output_path, blur_radius=5, max_words=20, font_size=48, text_align='justify'):
    """
    एक स्लाइड बनाता है जहां इमेज को ब्लर करके बैकग्राउंड में रखा जाता है
    और उसके ऊपर 20 शब्दों का टेक्स्ट दिखाया जाता है

    Parameters:
    -----------
    image_path : str
        इनपुट इमेज का पथ
    text : str
        पूरा टेक्स्ट जिससे 20 शब्द लेने हैं
    output_path : str
        आउटपुट स्लाइड को सेव करने का पथ
    blur_radius : int, optional
        बैकग्राउंड ब्लर की तीव्रता (डिफ़ॉल्ट: 5)
    max_words : int, optional
        प्रदर्शित करने के लिए अधिकतम शब्द (डिफ़ॉल्ट: 20)
    font_size : int, optional
        फॉन्ट का आकार (डिफ़ॉल्ट: 48)
    text_align : str, optional
        टेक्स्ट alignment - 'justify', 'center', 'left', 'right' (डिफ़ॉल्ट: 'justify')

    Returns:
    --------
    str : सेव की गई स्लाइड का पथ
    """

    # 1. पहले 20 शब्द निकालें
    words = text.split()[:max_words]
    display_text = " ".join(words)

    # 2. इमेज लोड करें
    try:
        img = Image.open(image_path)
    except Exception as e:
        print(f"इमेज लोड करने में त्रुटि: {e}")
        return None

    # 3. YouTube वीडियो साइज (1920x1080)
    youtube_size = (1920, 1080)

    # 4. इमेज को रिसाइज करें (aspect ratio बनाए रखते हुए)
    img_ratio = img.width / img.height
    target_ratio = youtube_size[0] / youtube_size[1]

    if img_ratio > target_ratio:
        # इमेज चौड़ी है, height के हिसाब से resize करें
        new_height = youtube_size[1]
        new_width = int(new_height * img_ratio)
    else:
        # इमेज लंबी है, width के हिसाब से resize करें
        new_width = youtube_size[0]
        new_height = int(new_width / img_ratio)

    img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

    # 5. Center crop करें
    left = (new_width - youtube_size[0]) // 2
    top = (new_height - youtube_size[1]) // 2
    right = left + youtube_size[0]
    bottom = top + youtube_size[1]
    img = img.crop((left, top, right, bottom))

    # 6. इमेज को ब्लर करें (बैकग्राउंड इफेक्ट)
    blurred_img = img.filter(ImageFilter.GaussianBlur(radius=blur_radius))

    # 7. ब्राइटनेस कम करें ताकि टेक्स्ट पढ़ने लायक हो
    enhancer = ImageEnhance.Brightness(blurred_img)
    darkened_img = enhancer.enhance(0.7)  # 70% brightness

    # 8. डार्क ओवरले ऐड करें
    overlay = Image.new('RGBA', youtube_size, (0, 0, 0, 120))  # Semi-transparent black
    final_img = darkened_img.convert('RGBA')
    final_img = Image.alpha_composite(final_img, overlay)

    # 9. टेक्स्ट ड्रा करने के लिए तैयार करें
    draw = ImageDraw.Draw(final_img)

    # 10. फॉन्ट लोड करें (Colab के लिए)
    try:
        # Colab में उपलब्ध फॉन्ट
        font_paths = [
            '/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf',
            '/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf',
            '/usr/share/fonts/truetype/noto/NotoSans-Bold.ttf',
            '/usr/share/fonts/truetype/msttcorefonts/Arial_Bold.ttf'
        ]

        font = None
        for font_path in font_paths:
            if os.path.exists(font_path):
                font = ImageFont.truetype(font_path, font_size)
                break

        if font is None:
            # कोई फॉन्ट नहीं मिला, default use करें
            font = ImageFont.load_default()
            print("डिफॉल्ट फॉन्ट का उपयोग किया जा रहा है")

    except Exception as e:
        print(f"फॉन्ट लोड करने में त्रुटि: {e}")
        font = ImageFont.load_default()

    # 11. टेक्स्ट को लपेटें (wrap)
    margin = 100
    max_width = youtube_size[0] - 2 * margin

    # मैन्युअल रूप से लाइनें तोड़ें (justify के लिए बेहतर control)
    words_list = display_text.split()
    lines = []
    current_line = []

    for word in words_list:
        test_line = ' '.join(current_line + [word])
        bbox = draw.textbbox((0, 0), test_line, font=font)
        line_width = bbox[2] - bbox[0]

        if line_width <= max_width:
            current_line.append(word)
        else:
            if current_line:
                lines.append(current_line)
            current_line = [word]

    # आखिरी line add करें
    if current_line:
        lines.append(current_line)

    # 12. टेक्स्ट की डाइमेंशन कैलकुलेट करें
    line_heights = []

    # Single space की width calculate करें (justify के लिए)
    space_bbox = draw.textbbox((0, 0), ' ', font=font)
    normal_space_width = space_bbox[2] - space_bbox[0]

    for line_words in lines:
        line_text = ' '.join(line_words)
        bbox = draw.textbbox((0, 0), line_text, font=font)
        line_height = bbox[3] - bbox[1]
        line_heights.append(line_height)

    total_height = sum(line_heights) + (10 * (len(lines) - 1))  # 10px line spacing

    # 13. टेक्स्ट की पोजीशन कैलकुलेट करें
    text_x = margin  # Left margin से start
    text_y = (youtube_size[1] - total_height) // 2  # Vertically centered

    # 14. Justify effect के साथ हर लाइन को ड्रा करें
    current_y = text_y
    for i, line_words in enumerate(lines):
        if text_align == 'justify' and i < len(lines) - 1 and len(line_words) > 1:
            # JUSTIFY: सभी lines को justify करें (आखिरी line को छोड़कर)
            # सभी शब्दों को बिना space के जोड़कर total width calculate करें
            words_without_spaces = ''.join(line_words)
            bbox = draw.textbbox((0, 0), words_without_spaces, font=font)
            words_total_width = bbox[2] - bbox[0]

            # Available space calculate करें
            available_space = max_width - words_total_width
            num_gaps = len(line_words) - 1

            # प्रत्येक gap की width
            if num_gaps > 0:
                gap_width = available_space / num_gaps
            else:
                gap_width = normal_space_width

            # शब्दों को ड्रा करें
            current_x = text_x
            for j, word in enumerate(line_words):
                draw.text((current_x, current_y), word, font=font, fill=(255, 255, 255, 255))

                # Word की width calculate करें
                word_bbox = draw.textbbox((0, 0), word, font=font)
                word_width = word_bbox[2] - word_bbox[0]

                # अगले word के लिए position
                current_x += word_width + gap_width

        elif text_align == 'center':
            # CENTER alignment
            line_text = ' '.join(line_words)
            bbox = draw.textbbox((0, 0), line_text, font=font)
            line_width = bbox[2] - bbox[0]
            line_x = (youtube_size[0] - line_width) // 2
            draw.text((line_x, current_y), line_text, font=font, fill=(255, 255, 255, 255))

        elif text_align == 'right':
            # RIGHT alignment
            line_text = ' '.join(line_words)
            bbox = draw.textbbox((0, 0), line_text, font=font)
            line_width = bbox[2] - bbox[0]
            line_x = youtube_size[0] - margin - line_width
            draw.text((line_x, current_y), line_text, font=font, fill=(255, 255, 255, 255))

        else:
            # LEFT alignment (या आखिरी line for justify)
            line_text = ' '.join(line_words)
            if text_align == 'justify':
                # आखिरी line को left align करें
                draw.text((text_x, current_y), line_text, font=font, fill=(255, 255, 255, 255))
            else:
                draw.text((text_x, current_y), line_text, font=font, fill=(255, 255, 255, 255))

        current_y += line_heights[i] + 10  # Move down for next line

    # 15. स्लाइड सेव करें
    try:
        # Ensure output directory exists
        output_dir = os.path.dirname(output_path)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Convert to RGB if saving as JPEG
        if output_path.lower().endswith('.jpg') or output_path.lower().endswith('.jpeg'):
            final_img = final_img.convert('RGB')

        final_img.save(output_path, quality=95)
        print(f"✅ स्लाइड सफलतापूर्वक सेव की गई: {output_path}")
        print(f"   आकार: {youtube_size[0]}x{youtube_size[1]}")
        print(f"   शब्द: {len(words)}/{max_words}")
        print(f"   ब्लर: {blur_radius}")
        print(f"   टेक्स्ट: {text_align.upper()} ALIGNED")

        return output_path

    except Exception as e:
        print(f"स्लाइड सेव करने में त्रुटि: {e}")
        return None

In [ ]:
# @title main_process()
def main_process(story_text, image_paths, audio_path, output_video="final_video.mp4"):
    """
    मुख्य प्रोसेस: स्टोरी टेक्स्ट, इमेज और ऑडियो से वीडियो बनाता है
    """

    # स्टोरी को 20 शब्दों के chunks में तोड़ें
    words = story_text.split()
    chunks = []
    max_words = 100;

    for i in range(0, len(words), max_words):
        chunk = " ".join(words[i:i+max_words])
        chunks.append(chunk)


    print(f"कुल {len(chunks)} स्लाइड बनेंगी")

    # स्लाइड इमेज बनाएं
    slide_images = []

    for i, chunk in enumerate(chunks):
        # अगर image_paths कम हों तो loop में उपयोग करें
        image_idx = i % len(image_paths)

        output_slide = f"slide_{i:03d}.jpg"

        # creating slides here
        slide_path = create_slide(
            image_path=image_paths,
            text=chunk,
            output_path=output_slide,
            blur_radius=2,
            max_words=max_words,
            font_size=48,
            text_align='justify'
        )

        slide_images.append(slide_path)
        print(f"स्लाइड {i+1} बन गई: {slide_path}")

    # वीडियो बनाएं (तेज़ तरीके से)
    print("\nवीडियो बनाना शुरू कर रहे हैं...")

    # प्रति स्लाइड अवधि (सेकंड में)
    # ऑडियो की लंबाई के हिसाब से अवधि calculate करें
    # पहले ऑडियो की लंबाई पता करें
    cmd = ['ffprobe', '-v', 'error', '-show_entries',
           'format=duration', '-of',
           'default=noprint_wrappers=1:nokey=1', audio_path]

    result = subprocess.run(cmd, capture_output=True, text=True)
    audio_duration = float(result.stdout.strip())

    duration_per_slide = audio_duration / len(slide_images)
    print(f"ऑडियो ड्यूरेशन: {audio_duration:.2f} सेकंड")
    print(f"प्रति स्लाइड: {duration_per_slide:.2f} सेकंड")

    # सबसे तेज़ वीडियो जनरेशन फंक्शन
    video_path = create_optimized_video(
        slide_images,
        audio_path,
        output_video,
        duration_per_slide
    )

    # Temporary स्लाइड इमेज साफ करें
    for slide in slide_images:
        if os.path.exists(slide):
            os.remove(slide)

    return video_path

In [ ]:
# @title entry point __name__
# उदाहरण डेटा
if __name__ == "__main__":

    # उदाहरण ऑडियो डाउनलोड करें (या अपना ऑडियो अपलोड करें)


    # Colab में फाइल अपलोड करने का तरीका
    from google.colab import files


    from pathlib import Path

    print("कृपया एक story फाइल अपलोड करें (txt )")
    text_file = files.upload()
    filename = list(text_file.keys())[0]
    file_path = Path(filename)

    # Saara text read karo
    story_text = file_path.read_text(encoding='utf-8')

    print("कृपया एक image फाइल अपलोड करें (jpg )")
    img_upload = files.upload()
    image_file = list(img_upload.keys())[0] if img_upload else None
    image_paths=[image_file]

    import os
    audio_file = '/content/drive/MyDrive/story.mp3'


    if os.path.exists(audio_file):
        print(f"ऑडियो फाइल get from driveअपलोड हुई: {audio_file}")

        # वीडियो बनाएं
        video_path = main_process(story_text, image_file, audio_file, "my_story_video.mp4")

        print(f"वीडियो तैयार है: {video_path}")

        print('clean audio file from google drive..')
        os.remove(audio_file);

        # print("वीडियो डाउनलोड करने के लिए:")
        # files.download(video_path)
    else:
        print("कृपया एक ऑडियो फाइल अपलोड करें (MP3 या WAV फॉर्मेट में)")
        uploaded = files.upload()
        audio_file = list(uploaded.keys())[0] if uploaded else None

        # start generating video
        video_path = main_process(story_text, image_file, audio_file, "my_story_video.mp4")

        print(f"वीडियो तैयार है: {video_path}")

# **Upload video to youtube channel**


In [ ]:

# @title YouTube Video Thumbnail Uploader - Google Colab
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client pillow -q
!pip install --upgrade google-api-python-client -q

In [ ]:
# @title 2. **Authentication Function (पहले वाले से)**

# import os
# import json
# from IPython.display import display, HTML
# from google.colab import files

# print("Google API Credentials के लिए:")
# print("1. https://console.developers.google.com/ पर जाएं")
# print("2. नया प्रोजेक्ट बनाएं")
# print("3. YouTube Data API v3 enable करें")
# print("4. OAuth 2.0 Client ID बनाएं (Desktop App चुनें)")
# print("5. JSON डाउनलोड करें और यहाँ अपलोड करें")

# # Credentials फ़ाइल अपलोड करें
# uploaded = files.upload()

# # अपलोड की गई फ़ाइल का नाम पाएं
# credentials_file = list(uploaded.keys())[0]
# print(f"Credentials फ़ाइल '{credentials_file}' अपलोड हुई")



import os
import pickle
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseUpload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.colab import files
import io
from PIL import Image
import requests

# YouTube API scopes (thumbnail upload के लिए)
SCOPES = [
    "https://www.googleapis.com/auth/youtube.upload",
    "https://www.googleapis.com/auth/youtube"
]

def get_youtube_service():
    """YouTube API service बनाता है"""
    credentials = None

    # Check if token exists
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)

    # If credentials are invalid, get new ones
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            # Upload credentials file
            print("Google API credentials JSON फ़ाइल अपलोड करें:")
            uploaded = files.upload()
            credentials_file = list(uploaded.keys())[0]

            flow = InstalledAppFlow.from_client_secrets_file(
                credentials_file, SCOPES)
            credentials = flow.run_local_server(port=0)

        # Save credentials
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)

    return build('youtube', 'v3', credentials=credentials)

In [ ]:
# @title 3. **थंबनेल अपलोड Function**

def upload_thumbnail(youtube, video_id, thumbnail_path):
    """
    YouTube वीडियो के लिए थंबनेल अपलोड करें

    Parameters:
    - youtube: YouTube API service object
    - video_id: YouTube video ID
    - thumbnail_path: थंबनेल इमेज का पथ
    """

    try:
        # थंबनेल फ़ाइल की जाँच करें
        if not os.path.exists(thumbnail_path):
            print(f"❌ थंबनेल फ़ाइल नहीं मिली: {thumbnail_path}")
            return False

        # थंबनेल specifications check करें
        try:
            with Image.open(thumbnail_path) as img:
                width, height = img.size
                print(f"📊 थंबनेल डाइमेंशन: {width}x{height}")

                # Recommended size: 1280x720 (16:9 ratio)
                if width < 640 or height < 480:
                    print("⚠️  Warning: थंबनेल बहुत छोटा है (minimum 640x480 recommended)")

                # File format check
                if img.format not in ['JPEG', 'JPG', 'PNG', 'GIF', 'BMP']:
                    print("⚠️  Warning: YouTube केवल JPEG, PNG, GIF, BMP सपोर्ट करता है")
        except Exception as e:
            print(f"⚠️  इमेज check में error: {e}")

        # थंबनेल अपलोड करें
        print(f"🖼️  थंबनेल अपलोड हो रहा है...")

        with open(thumbnail_path, 'rb') as thumbnail_file:
            media = MediaIoBaseUpload(
                thumbnail_file,
                mimetype='image/jpeg' if thumbnail_path.lower().endswith(('.jpg', '.jpeg')) else 'image/png',
                resumable=True
            )

            request = youtube.thumbnails().set(
                videoId=video_id,
                media_body=media
            )

            response = request.execute()

            print("✅ थंबनेल सफलतापूर्वक अपलोड हुआ!")
            print(f"   Size: {response.get('size', 'N/A')}")
            print(f"   Video ID: {video_id}")

            return True

    except Exception as e:
        print(f"❌ थंबनेल अपलोड में error: {str(e)}")
        return False

In [ ]:
# @title 4. **वीडियो अपलोड + थंबनेल Function**

def upload_video_with_thumbnail(video_path, thumbnail_path, video_details):
    """
    वीडियो और थंबनेल एक साथ अपलोड करें

    Parameters:
    - video_path: वीडियो फ़ाइल का पथ
    - thumbnail_path: थंबनेल फ़ाइल का पथ
    - video_details: dictionary with title, description, etc.
    """

    youtube = get_youtube_service()

    # पहले वीडियो अपलोड करें
    print("🎥 वीडियो अपलोड शुरू...")

    # वीडियो metadata
    body = {
        "snippet": {
            "title": video_details.get('title', 'My Video'),
            "description": video_details.get('description', ''),
            "tags": video_details.get('tags', []),
            "categoryId": video_details.get('category_id', '22')
        },
        "status": {
            "privacyStatus": video_details.get('privacy_status', 'private'),
            "selfDeclaredMadeForKids": False
        }
    }

    # वीडियो अपलोड
    try:
        media = MediaFileUpload(
            video_path,
            mimetype='video/*',
            resumable=True
        )

        request = youtube.videos().insert(
            part=",".join(body.keys()),
            body=body,
            media_body=media
        )

        video_response = request.execute()
        video_id = video_response['id']

        print(f"✅ वीडियो अपलोड हुआ! Video ID: {video_id}")
        print(f"📺 URL: https://www.youtube.com/watch?v={video_id}")

        # थंबनेल अपलोड करें
        if thumbnail_path and os.path.exists(thumbnail_path):
            # थोड़ा wait करें (YouTube processing के लिए)
            import time
            print("⏳ वीडियो processing का इंतज़ार...")
            time.sleep(10)

            # थंबनेल अपलोड
            thumbnail_success = upload_thumbnail(youtube, video_id, thumbnail_path)

            if thumbnail_success:
                print("🎉 वीडियो और थंबनेल दोनों सफलतापूर्वक अपलोड हुए!")
            else:
                print("⚠️  वीडियो अपलोड हुआ, लेकिन थंबनेल में समस्या")
        else:
            print("⚠️  थंबनेल फ़ाइल नहीं मिली, सिर्फ वीडियो अपलोड हुआ")

        return video_id

    except Exception as e:
        print(f"❌ वीडियो अपलोड में error: {str(e)}")
        return None

In [ ]:
# @title authenticate_colab()
# Clear any existing tokens
!rm -f token.pickle 2>/dev/null || true

# %%
import os
import pickle
import json
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseUpload
from google_auth_oauthlib.flow import Flow
from google.auth.transport.requests import Request
from google.colab import files
from PIL import Image
import requests
import time


# ## 1. **Colab-Friendly Authentication (No Browser)**

def authenticate_colab():
    """
    Colab के लिए optimized authentication - manual code entry
    """
    # SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]

    SCOPES = [
    "https://www.googleapis.com/auth/youtube.upload",
    "https://www.googleapis.com/auth/userinfo.email",
    "https://www.googleapis.com/auth/userinfo.profile",
    "openid"
]


    # Check for existing token
    if os.path.exists('token.pickle'):
        try:
            with open('token.pickle', 'rb') as token:
                credentials = pickle.load(token)

            # Check if token is expired
            if credentials and credentials.expired and credentials.refresh_token:
                credentials.refresh(Request())
                print("✅ Token refreshed successfully")
                with open('token.pickle', 'wb') as token:
                    pickle.dump(credentials, token)

            if credentials and credentials.valid:
                print("✅ Using existing credentials")
                return build('youtube', 'v3', credentials=credentials)

        except Exception as e:
            print(f"⚠️ Error loading token: {e}")

    # New authentication required
    print("\n" + "="*60)
    print("🔐 NEW AUTHENTICATION REQUIRED")
    print("="*60)

    # Step 1: Upload credentials
    print("\n📁 STEP 1: Upload your Google API credentials JSON file")
    print("(The file you downloaded from Google Cloud Console)")

    uploaded = files.upload()

    if not uploaded:
        print("❌ No file uploaded. Please upload credentials.json")
        return None

    creds_file = list(uploaded.keys())[0]
    print(f"✅ File uploaded: {creds_file}")

    # Step 2: Create OAuth flow with out-of-band (OOB) redirect
    try:
        flow = Flow.from_client_secrets_file(
            creds_file,
            scopes=SCOPES,
            redirect_uri='urn:ietf:wg:oauth:2.0:oob'  # OOB for manual entry
        )

        # Generate authorization URL
        auth_url, _ = flow.authorization_url(
            access_type='offline',
            prompt='consent',
            include_granted_scopes='true'
        )

        # Display instructions
        print("\n" + "="*60)
        print("📋 STEP 2: Get Authorization Code")
        print("="*60)
        print("\n1. Click this link (open in new tab):")
        print(f"\n🔗 {auth_url}")
        print("\n2. Login with your Google account")
        print("3. Click 'Allow' to grant permissions")
        print("4. You will see a code on the screen")
        print("5. Copy that code and paste it below")
        print("\n" + "="*60)

        # Get code from user
        auth_code = input("\n📝 Paste the authorization code here: ").strip()

        if not auth_code:
            print("❌ No code provided")
            return None

        # Step 3: Exchange code for tokens
        print("\n🔄 STEP 3: Exchanging code for access token...")

        flow.fetch_token(code=auth_code)
        credentials = flow.credentials

        # Save credentials for future
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)

        print("✅ Authentication successful!")
        print("✅ Token saved for future use")

        return build('youtube', 'v3', credentials=credentials)

    except Exception as e:
        print(f"❌ Authentication failed: {str(e)}")
        return None


# # ## 2. **Test Authentication**


# # Test authentication
# print("Testing authentication...")
# youtube = authenticate_colab()

# if youtube:
#     print("\n🎉 Authentication successful! YouTube API ready to use.")
# else:
#     print("\n❌ Authentication failed. Please check your credentials.")

In [ ]:
# @title 5. **मुख्य Execution Function**

def main():
    """
    मुख्य function - यूज़र के लिए interactive
    """
    print("="*60)
    print("YouTube Video & Thumbnail Uploader")
    print("="*60)

    try:
        # YouTube service initialize करें
        # youtube = get_youtube_service()
        youtube = authenticate_colab()


        #created above
        video_file="my_story_video.mp4"

        if not video_file:
             print("❌ कोई वीडियो फ़ाइल अपलोड नहीं हुई")
             return

        print("\n🖼️  थंबनेल फ़ाइल अपलोड करें:")
        thumbnail_uploaded = files.upload()
        thumbnail_file = list(thumbnail_uploaded.keys())[0] if thumbnail_uploaded else None

        # Video details input
        # title = input("\n📝 वीडियो का शीर्षक: ") or "My Video"
        # description = input("📝 वीडियो का विवरण: ") or ""

        # privacy = input("\n🔒 Privacy (private/unlisted/public) [default: private]: ") or "private"


        # video_details = {
        #         'title': title,
        #         'description': description,
        #         'privacy_status': privacy,
        #         'category_id': '27',
        #         'tags': ['youtube', 'api', 'python']
        #     }

        import json
        print('upload video details file (.json)')
        details_upload = files.upload()
        video_details_file = list(details_upload.keys())[0] if details_upload else None
        with open(video_details_file, "r", encoding="utf-8") as f:
              video_details = json.load(f)

            # Upload video with thumbnail
        video_id = upload_video_with_thumbnail(video_file, thumbnail_file, video_details)


    except Exception as e:
        print(f"\n❌ Error: {str(e)}")



# start execution here
if __name__ == "__main__":
    main()